In [29]:
using Test

function count_information(vector)
    print("length of vector is: \t", length(vector), " so half is:\t", trunc(Int, length(vector)/2))
    max_mersenne_layers             = trunc(Int, log2(length(vector)/2))-1
    print("\nmax layers", max_mersenne_layers)
    cases_on_near_side              = trunc(Int, length(vector)/2)
    cover_on_near_side              = 0
    mersenne_layers_on_near_side    = 1
    
    while cover_on_near_side + (2^mersenne_layers_on_near_side)-1 <= cases_on_near_side
        mersenne_layers_on_near_side += 1
        cover_on_near_side += (2^mersenne_layers_on_near_side)-1
    end
    
    @test cover_on_near_side <= cases_on_near_side
    
    alpha           = vector[1]
    max_info        = 0
    actual_info     = 0
    for m in 1:mersenne_layers_on_near_side
        max_info = max_info + 1-(1/(2^m))
        print(" max", max_info)
    end
    
    for m in 1:mersenne_layers_on_near_side
        first = Int(   (2^(m-1))-1  )
        last  = Int(   (2^ m   )-1  )
        if last == 1
            this_slice = [vector[last]]
            deleteat!(vector, last)
            actual_info = actual_info + 1-(1/(2^m))
        else
            this_slice = vector[first:last]
            deleteat!(vector, first:last)
            if this_slice[1] == alpha
                if all(y->y==this_slice[1],this_slice)
                    actual_info = actual_info + 1-(1/(2^m))
                end
            end
        end
        print(" actual", actual_info)
    end
    println("\nactual info:\t", actual_info)
    println("\nmax info:\t",    max_info)
    information_count = actual_info/max_info
    return information_count
end

function test_count_information()
    references = [  [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 0.23529411764705882],
                    [[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,], 1.0],                          
                    [[2,2,2,2,1,1,1,1],                     1.0], 
                    [[1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2],     1.0],                          
                    [[2,1,2,1,2,1,2,1,2,1,2,1,2,1],         0.5882352941176471], 
                    [[2,1,2,1,2,1,2,1,2,1,2,1,2,1],         0.5882352941176471],                          
                    [[1,2,1,2,1,2,1,2,1,2,1,2,1,1],         0.0], 
                    [[1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2], 1.0],                          
                    [[2,2],                                 1.0],
                    
                    [[1,2,1,2,1,2,1,2],                     0.0],                
                    [[2,1,2,1,2,1,2,1],                     0.0], 
        
                    [[1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2],     0.0],
                    [[2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1],     0.0], 
                    [[2,1,1,1,2,2,2,2,1,1,1,1,2,2,2,1],     0.0],
                    [[1,2,2,2,1,1,1,1,2,2,2,2,1,1,1,2],     0.0], 
        
                    [[1,1,2,1,2,1,2,2],                     0.0],                
                    [[2,2,1,2,1,2,1,1],                     0.0], 
                    [[2,1,1,1,1,2,2,2],                     0.0],                  
                    [[1,2,2,2,2,1,1,1],                     0.0], 
                    [[1,2,1,1,2,2],                         0.0],                    
                    [[2,2,1,2,1,1],                         0.0], 
                    [[1,1,1,2,2,2],                         0.0],                      
                    [[2,2,2,1,1,1],                         0.0]
                                                                                                    ]
    for ref in references
        vector = ref[1]
        print("\n\nordered cases in space from origin:\t", vector)
        information_ref   = ref[2]
        print("\nreference information ratio:\t\t", information_ref)
        
        information_count = count_information(vector)
        print("\nderived information ratio:\t\t", information_count)
        
        @test information_count==information_ref
    end
end

test_count_information()




ordered cases in space from origin:	[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
reference information ratio:		0.23529411764705882length of vector is: 	20 so half is:	10
max layers2 max0.5 max1.25 max2.125 actual0.5 actual0.5 actual0.5
actual info:	0.5

max info:	2.125

derived information ratio:		0.23529411764705882

ordered cases in space from origin:	[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
reference information ratio:		1.0length of vector is: 	40 so half is:	20
max layers3Test Failed at In[29]:16
  Expression: cover_on_near_side <= cases_on_near_side
   Evaluated: 25 <= 20


LoadError: [91mThere was an error during testing[39m